# `MEDUSA`
aka. Dynamic-Prospective LCA aka. Union(premise, temporalis)

In [4]:
from bw_temporalis import easy_timedelta_distribution, TemporalDistribution, easy_datetime_distribution
import sys
import os
sys.path.append(os.path.realpath('../'))
from medusa.edge_extractor import *
from medusa.matrix_modifier import *
from medusa.medusa_lca import *
from medusa.timeline_builder import *
import bw2data as bd
import bw2calc as bc
import numpy as np
import pandas as pd

In [1]:
from tests.db_electrolysis import db_electrolysis
db_electrolysis()

100%|██████████| 1/1 [00:00<00:00, 8473.34it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 44384.17it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 45343.83it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 3/3 [00:00<00:00, 76725.07it/s]

Vacuuming database 


In [2]:
demand = {('foreground', 'heat_from_hydrogen'): 1}
gwp = ('GWP', 'example')

# Static LCA

In [5]:
slca = bc.LCA(demand, gwp)
slca.lci()
slca.lcia()
print(f'Static LCA score: {slca.score}')

Static LCA score: 40.94999821782114


# `MEDUSA` LCA

A MEDUSA LCA builds upon a static LCA, but adds a temporal dimensions, linking to prospective LCA databases. Similarly to a `Temporalis LCA`, the supply chain graph is traversed, taking into account temporal distributions of the edges. 

For now, only the foreground system is assumed to have temporal distributions. Therefore, we define a filter function, that tells to EdgeExtracter (which is doing the actual graph traversal and saves the edges with respective timestamps), when a database that is known to have no temporal distributions (i.e., the prospective background databases) is reached, so that the traversal can be stopped.

In [6]:
SKIPPABLE = [node.id for node in bd.Database('background_2020')] + [
    node.id for node in bd.Database('background_2024')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

Now we can do the graph traversal and create a timeline of edges:

In [7]:
eelca = EdgeExtractor(slca, edge_filter_function=filter_function)
timeline = eelca.build_edge_timeline()

Starting graph traversal
Calculation count: 4


Next, we define a dictionary containing the dates of our prospective background databases. Using this, we can create a timeline dataframe. 

The dates of the edges are mapped to the prospective background databases; interpolation is used for dates in between the dates of the background databases. The default is linear interpolation, another currently included option is "nearest", choosing the next best fitting database.

In [8]:
timeline

[Edge(distribution=TemporalDistribution instance with 1 values and total: 1, leaf=False, consumer=-1, producer=8, td_producer=1, td_consumer=TemporalDistribution instance with 1 values and total: 1, abs_td_producer=TemporalDistribution instance with 1 values and total: 1, abs_td_consumer=None),
 Edge(distribution=TemporalDistribution instance with 2 values and total: 0.7, leaf=False, consumer=8, producer=7, td_producer=TemporalDistribution instance with 2 values and total: 0.7, td_consumer=TemporalDistribution instance with 1 values and total: 1, abs_td_producer=TemporalDistribution instance with 2 values and total: 0.7, abs_td_consumer=TemporalDistribution instance with 1 values and total: 1),
 Edge(distribution=TemporalDistribution instance with 2 values and total: 9.1, leaf=False, consumer=7, producer=6, td_producer=TemporalDistribution instance with 1 values and total: 13, td_consumer=TemporalDistribution instance with 2 values and total: 0.7, abs_td_producer=TemporalDistribution i

In [9]:
database_date_dict = {
            datetime.strptime("2020", "%Y"): 'background_2020',
            datetime.strptime("2024", "%Y"): 'background_2024',
        }

timeline_df = create_grouped_edge_dataframe(timeline, database_date_dict, interpolation_type="linear")
timeline_df

All databases in database_date_dict exist as brightway project databases


,hash_producer,date_producer,producer,producer_name,hash_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,2018,2018-01-01,2,Electricity mix,2019,2019-01-01,6,"Hydrogen production, electrolysis",1.0,{'background_2020': 1}
1,2019,2019-01-01,2,Electricity mix,2019,2019-01-01,6,"Hydrogen production, electrolysis",3.0,{'background_2020': 1}
2,2019,2019-01-01,6,"Hydrogen production, electrolysis",2022,2022-01-01,7,"Hydrogen production, electrolysis2",13.0,{'background_2020': 1}
3,2020,2020-01-01,2,Electricity mix,2019,2019-01-01,6,"Hydrogen production, electrolysis",1.0,{'background_2020': 1}
4,2020,2020-01-01,2,Electricity mix,2021,2021-01-01,6,"Hydrogen production, electrolysis",1.0,{'background_2020': 1}
5,2021,2021-01-01,2,Electricity mix,2021,2021-01-01,6,"Hydrogen production, electrolysis",3.0,"{'background_2020': 0.7494866529774127, 'backg..."
6,2021,2021-01-01,6,"Hydrogen production, electrolysis",2024,2024-01-01,7,"Hydrogen production, electrolysis2",13.0,"{'background_2020': 0.7494866529774127, 'backg..."
7,2022,2022-01-01,2,Electricity mix,2021,2021-01-01,6,"Hydrogen production, electrolysis",1.0,"{'background_2020': 0.4996577686516085, 'backg..."
8,2022,2022-01-01,7,"Hydrogen production, electrolysis2",2024,2024-01-01,8,"Heat production, hydrogen",0.63,"{'background_2020': 0.4996577686516085, 'backg..."
9,2024,2024-01-01,7,"Hydrogen production, electrolysis2",2024,2024-01-01,8,"Heat production, hydrogen",0.07,{'background_2024': 1}


Now, we want to create a datapackage that takes care of relinking processes to our prospective databases. To do so, we need to provide the timeline dataframe, the dict of prospective databases and corresponding years, and a new dictionary that defines at which point in time our functional unit is assessed *(We can probably include this information in the database_date_dict in the future, but for now, this works)*.

In [10]:
demand_timing_dict = create_demand_timing_dict(timeline_df, demand)

dp = create_datapackage_from_edge_timeline(timeline_df, database_date_dict, demand_timing_dict)

Finally, we just have to reformat our input data for the LCA, add our datapackage containing the patches, and run the lca.

In [11]:
fu, data_objs, remapping = prepare_medusa_lca_inputs(demand=demand, demand_timing_dict=demand_timing_dict, method=gwp) 
lca = bc.LCA(fu, data_objs = data_objs + [dp], remapping_dicts=remapping)
lca.lci()
lca.lcia()

Let's take a look at the results:

In [12]:
print('New MEDUSA LCA Score:', lca.score)
print('Old static LCA Score:', slca.score)

New MEDUSA LCA Score: 54.258237303744316
Old static LCA Score: 40.94999821782114


In [13]:
df = pd.DataFrame(lca.technosphere_matrix.toarray())
df.rename(lca.dicts.activity.reversed, inplace=True, axis=0)
df.rename(lca.dicts.activity.reversed, inplace=True, axis=1)
df


,2,3,4,5,6,7,8,2002018,2002019,2002020,2002021,2002022,6002019,6002021,7002022,7002024,8002024
2,1.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,-0.250513,-0.500342,0.0,0.0,0.0,0.0,0.00
3,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-0.749487,-0.499658,0.0,0.0,0.0,0.0,0.00
5,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00
6,0.0,0.0,0.0,0.0,1.0,-13.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00
7,0.0,0.0,0.0,0.0,0.0,1.0,-0.7,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00
8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00
2002018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,-1.0,0.0,0.0,0.0,0.00
2002019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,-3.0,0.0,0.0,0.0,0.00
2002020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,-1.0,-1.0,0.0,0.0,0.00


In [ ]:
new_edges_df.explode(['consumer_date', 'producer_date', 'amount'])

NameError: name 'new_edges_df' is not defined

In [ ]:
rem = lca.remapping_dicts['activity']
rem

{1: ('temporalis-bio', 'CO2'),
 2: ('background_2024', 'electricity_mix'),
 3: ('background_2024', 'electricity_wind'),
 4: ('background_2020', 'electricity_mix'),
 5: ('background_2020', 'electricity_wind'),
 6: ('foreground', 'someotherprocess'),
 7: ('foreground', 'electrolysis'),
 8: ('foreground', 'heat_from_hydrogen')}

In [ ]:
dict(lca.dicts.activity)

{2: 0,
 3: 1,
 4: 2,
 5: 3,
 6: 4,
 7: 5,
 8: 6,
 2002021: 7,
 2002022: 8,
 2002023: 9,
 2002024: 10,
 6002022: 11,
 6002024: 12,
 7002022: 13,
 7002024: 14,
 8002024: 15}